# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4768cb0910>
├── 'a' --> tensor([[ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873]])
└── 'x' --> <FastTreeValue 0x7f4768cb0490>
    └── 'c' --> tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                        [-0.6430, -1.1469,  1.1182, -0.4121],
                        [ 0.4835, -1.2793,  1.1690, -0.3093]])

In [4]:
t.a

tensor([[ 0.8514,  1.1174,  2.9507],
        [ 0.1847, -0.1599,  0.6873]])

In [5]:
%timeit t.a

58.4 ns ± 0.0845 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4768cb0910>
├── 'a' --> tensor([[ 0.1911,  0.3423,  0.3140],
│                   [ 0.6817,  0.9405, -0.0683]])
└── 'x' --> <FastTreeValue 0x7f4768cb0490>
    └── 'c' --> tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                        [-0.6430, -1.1469,  1.1182, -0.4121],
                        [ 0.4835, -1.2793,  1.1690, -0.3093]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.0364 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8514,  1.1174,  2.9507],
               [ 0.1847, -0.1599,  0.6873]]),
    x: Batch(
           c: tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8514,  1.1174,  2.9507],
        [ 0.1847, -0.1599,  0.6873]])

In [11]:
%timeit b.a

60.2 ns ± 0.046 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.8818,  1.3941, -1.7679],
               [-0.3585,  0.4254, -0.5029]]),
    x: Batch(
           c: tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 1.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 34.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 362 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 424 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f46aeeb3130>
├── 'a' --> tensor([[[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]],
│           
│                   [[ 0.8514,  1.1174,  2.9507],
│                    [ 0.1847, -0.1599,  0.6873]]])
└── 'x' --> <FastTreeValue 0x7f46c057c310>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 63.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f46aeeb3eb0>
├── 'a' --> tensor([[ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873],
│                   [ 0.8514,  1.1174,  2.9507],
│                   [ 0.1847, -0.1599,  0.6873]])
└── 'x' --> <FastTreeValue 0x7f46aee97400>
    └── 'c' --> tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                        [-0.6430, -1.1469,  1.1182, -0.4121],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5873, -1.2800,  1.8497,  1.2178],
                       [-0.6430, -1.1469,  1.1182, -0.4121],
                       [ 0.4835, -1.2793,  1.1690, -0.3093]],
              
                      [[ 0.5873, -1.2800,  1.8497,  1.2178],
                       [-0.6430, -1.1469,  1.1182, -0.4121],
                       [ 0.4835, -1.2793,  1.1690, -0.3093]],
              
                      [[ 0.5873, -1.2800,  1.8497,  1.2178],
                       [-0.6430, -1.1469,  1.1182, -0.4121],
                       [ 0.4835, -1.2793,  1.1690, -0.3093]],
              
                      [[ 0.5873, -1.2800,  1.8497,  1.2178],
                       [-0.6430, -1.1469,  1.1182, -0.4121],
                       [ 0.4835, -1.2793,  1.1690, -0.3093]],
              
                      [[ 0.5873, -1.2800,  1.8497,  1.2178],
                       [-0.6430, -1.1469,  1.1182, -0.4121],
                       [ 0.4835, -1.2793,  1.1690, -0.3093]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 176 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093],
                      [ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093],
                      [ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093],
                      [ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093],
                      [ 0.5873, -1.2800,  1.8497,  1.2178],
                      [-0.6430, -1.1469,  1.1182, -0.4121],
                      [ 0.4835, -1.2793,  1.1690, -0.3093],
                      [ 0.5873, -1.2800,  1.8497,  1.2178],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
